In [1]:
import re
import pandas as pd
pd.options.display.max_columns = 9
pd.options.display.max_rows = 3
np = pd.np
np.norm = np.linalg.norm
from datetime import datetime, date
import json
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
from sklearn.feature_extraction.text import TfidfVectorizer  # equivalent to TFIDFTransformer(CountVectorizer())
from django.db.models import Sum
from pacs.models import CampaignDetail, WorkingTransactions
import django
django.setup()
CampaignDetail.objects.count(), WorkingTransactions.objects.count()

(3280, 997955)

# Django
This is how you join CampaignDetail & WorkingTransactions  
and aggregate the WorkingTransactions.amount

In [2]:
qs = CampaignDetail.objects.annotate(net_amount=Sum('workingtransactions__amount')).values().all()
print('Net transactions: {:,}M'.format(round(sum(qs.values_list('net_amount', flat=True)) / 1e6)))

Net transactions: 1,039.0M


Convert a Django Queryset into a Pandas DataFrame

In [3]:
df = pd.DataFrame.from_records(qs)
df.columns

Index([u'candidate_name', u'committee_name', u'committee_subtype',
       u'committee_type', u'db_update_status', u'election', u'filer_id',
       u'grassroots', u'instate', u'net_amount', u'num_transactions', u'party',
       u'phone', u'race', u'total', u'total_spent', u'website'],
      dtype='object')

In [4]:
df = df[df.committee_name.astype(bool)].copy()
df

,candidate_name,committee_name,committee_subtype,committee_type,...,race,total,total_spent,website
0,Katherine Schacht,Supporters of Katherine Schacht,None,CC,...,2016 Election Director Emerald People's Utilit...,1083.15,1664.61,None
...,...,...,...,...,...,...,...,...,...
3278,Cottage Grove Blackberry Pie Society,Cottage Grove Blackberry Pie Society,Miscellaneous,PAC,...,,180.00,171.80,None


# Pandas DataFrame.join
What if you only want positive transactions?

In [5]:
qs_pos = CampaignDetail.objects.filter(committee_name__isnull=False, workingtransactions__amount__gt=0)
qs_pos = qs_pos.annotate(pos_amount=Sum('workingtransactions__amount'))
df_pos = df.join(pd.DataFrame.from_records(qs_pos.values('pos_amount').all())['pos_amount'])
df_pos

,candidate_name,committee_name,committee_subtype,committee_type,...,total,total_spent,website,pos_amount
0,Katherine Schacht,Supporters of Katherine Schacht,None,CC,...,1083.15,1664.61,None,8104.57
...,...,...,...,...,...,...,...,...,...
3278,Cottage Grove Blackberry Pie Society,Cottage Grove Blackberry Pie Society,Miscellaneous,PAC,...,180.00,171.80,None,NaN


### What if I just insert a new column with the values?

In [6]:
df = pd.DataFrame.from_records(qs)
df = pd.DataFrame(df[df.committee_name.astype(bool)])
df['pos_amount'] = pd.DataFrame.from_records(qs_pos.values('pos_amount').all())['pos_amount']
df

,candidate_name,committee_name,committee_subtype,committee_type,...,total,total_spent,website,pos_amount
0,Katherine Schacht,Supporters of Katherine Schacht,None,CC,...,1083.15,1664.61,None,8104.57
...,...,...,...,...,...,...,...,...,...
3278,Cottage Grove Blackberry Pie Society,Cottage Grove Blackberry Pie Society,Miscellaneous,PAC,...,180.00,171.80,None,NaN


# Pandas indices are tricky
Did all the rows get inserted in the right place (are the indices still alligned)


In [7]:
df == df_pos

,candidate_name,committee_name,committee_subtype,committee_type,...,total,total_spent,website,pos_amount
0,True,True,False,True,...,True,True,False,True
...,...,...,...,...,...,...,...,...,...
3278,True,True,True,True,...,True,True,False,False


In [8]:
pd.options.display.max_rows = 6
(df == df_pos).mean()

candidate_name       1.000000
committee_name       1.000000
committee_subtype    0.350671
                       ...   
total_spent          0.946309
website              0.037752
pos_amount           0.730705
dtype: float64

# A NaN is not equal to a NaN!
Any operation involving a NaN returns a NaN  
And NaN (like None) always evaluates to False  

In [9]:
(df == df_pos).mean() + df.isnull().mean()

candidate_name       1
committee_name       1
committee_subtype    1
                    ..
total_spent          1
website              1
pos_amount           1
dtype: float64

# Negative transaction amounts?

In [10]:
qs_neg = CampaignDetail.objects.filter(workingtransactions__amount__lt=0)
qs_neg = qs_neg.annotate(neg_amount=Sum('workingtransactions__amount'))
df = df.join(pd.DataFrame.from_records(qs_neg.values('neg_amount').all())['neg_amount'])
df

,candidate_name,committee_name,committee_subtype,committee_type,...,total_spent,website,pos_amount,neg_amount
0,Katherine Schacht,Supporters of Katherine Schacht,None,CC,...,1664.61,None,8104.57,-145.06
1,James Diefenderfer,James Diefenderfer Political Action Committee,None,CC,...,4616.00,None,9968.10,-250.54
2,Josephine County Democratic Central Committee,Josephine County Democratic Central Committee,Political Party,PAC,...,21791.93,None,49027.07,-1205.29
...,...,...,...,...,...,...,...,...,...
3276,American College of Cardiology - Oregon Chapte...,American College of Cardiology - Oregon Chapte...,Miscellaneous,PAC,...,5999.39,None,NaN,NaN
3277,Oregon Professional Fire Fighters PAC,Oregon Professional Fire Fighters PAC,Miscellaneous,PAC,...,17500.00,None,NaN,NaN
3278,Cottage Grove Blackberry Pie Society,Cottage Grove Blackberry Pie Society,Miscellaneous,PAC,...,171.80,None,NaN,NaN


In [11]:
print('Positve transactions: {:,} M'.format(round(sum(qs_pos.values_list('pos_amount', flat=True)) / 1e6)))

Positve transactions: 933.0 M


In [12]:
print('Negative transactions: {:,} M'.format(round(sum(qs_neg.values_list('neg_amount', flat=True)) / 1.e6, 2)))

Negative transactions: -0.42 M


In [13]:
print('Net net transactions: {:,} M'.format(round(sum(qs.values_list('net_amount', flat=True)) / 1.e6)))

Net net transactions: 1,039.0 M


## Something's fishy in Denmark ^

In [14]:
df.sum()

filer_id       3.072790e+07
grassroots     6.504941e+02
instate        1.397825e+03
                   ...     
total_spent    3.014980e+08
pos_amount     6.883204e+08
neg_amount    -3.254632e+05
dtype: float64

In [20]:
print('Net amount: ${:} M'.format(round(df.sum()[['pos_amount', 'neg_amount']].sum()/1e6, 2)))

Net amount: $687.99 M


In [21]:
print('Volume: ${:} M'.format(round(np.abs(df.sum()[['pos_amount', 'neg_amount']]).sum()/1e6, 2)))

Volume: $688.65 M


# Directed graph of financial transactions
Are the payee_committee_ids the same as "filer_id"?

In [22]:
filer_id = set(pd.DataFrame.from_records(WorkingTransactions.objects.values(
               'filer_id').all()).dropna().values.T[0])
payee_id = set(pd.DataFrame.from_records(WorkingTransactions.objects.values(
               'contributor_payee_committee_id').all()).dropna().values.T[0])
com_id = set()
len(payee_id.intersection(filer_id)) * 1. / len(filer_id)

0.5310975609756098

# Good enough for Government Work
53% of payee_ids were found in the filer_id of the same Table
filer_id -> payee_id

In [23]:
qs = WorkingTransactions.objects.filter(filer_id__isnull=False, 
                                        contributor_payee_committee_id__isnull=False,
                                        amount__gt=0)
df_trans = pd.DataFrame.from_records(qs.values().all())
_, trans = df_trans.iterrows().next()
print(trans)
print(trans.index.values)

addr_line1    1149 Court St NE
addr_line2                None
amount                    5000
                    ...       
tran_date           2012-08-23
tran_id                1317714
zip                      97301
Name: 0, dtype: object
['addr_line1' 'addr_line2' 'amount' 'book_type' 'city' 'contributor_payee'
 'contributor_payee_class' 'contributor_payee_committee_id' 'direction'
 'filed_date' 'filer' u'filer_id_id' 'purp_desc' 'purpose_codes' 'state'
 'sub_type' 'tran_date' 'tran_id' 'zip']


# Lets compute a similarity matrix from positive transactions

In [34]:
ids = [int(i) for i in payee_id.intersection(filer_id)]
id_set = set(ids)
id_str = [str(int(i)) for i in ids]
N = len(ids)
cov = pd.DataFrame(np.zeros((N, N)),
                   index=pd.Index(id_str, name='payee'),
                   columns=pd.Index(id_str, name='filer'))
print(cov)
for rownum, trans in df_trans.iterrows():
    fid = trans['filer_id_id']
    # print(trans.index.values)
    cid = trans['contributor_payee_committee_id']
    if fid in id_set and cid in id_set:
#         if not (fid % 100):
#             print(cov[str(fid)][str(cid)])
        #only populate the upper
        if fid > cid:
            fid, cid = cid, fid
        amount = abs(trans['amount'])
        if amount > 0:
            cov[str(fid)][str(cid)] += amount
cov.describe()
    

filer  3  4  9  10  ...    16368  16371  16372  16373
payee               ...                              
3      0  0  0   0  ...        0      0      0      0
4      0  0  0   0  ...        0      0      0      0
9      0  0  0   0  ...        0      0      0      0
...   .. .. ..  ..  ...      ...    ...    ...    ...
16371  0  0  0   0  ...        0      0      0      0
16372  0  0  0   0  ...        0      0      0      0
16373  0  0  0   0  ...        0      0      0      0

[1742 rows x 1742 columns]


,3,4,9,10,...,16368,16371,16372,16373
count,1742.000000,1742.000000,1742.000000,1742.000000,...,1742.000000,1742,1742,1742
mean,508.514259,1453.846768,6.601607,1685.432233,...,0.904311,0,0,0
std,3163.788917,12354.042222,104.601433,15011.489409,...,32.310260,0,0,0
...,...,...,...,...,...,...,...,...,...
50%,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0,0
75%,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0,0
max,54000.000000,318000.000000,3250.000000,504000.000000,...,1325.310000,0,0,0


In [30]:
cov

filer,3,4,9,10,...,16368,16371,16372,16373
payee,,,,,,,,,
3,0,0,0,0,...,0.00,0,0,0
4,0,0,0,0,...,0.00,0,0,0
9,0,0,0,0,...,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...
16371,0,0,0,0,...,250.00,0,0,0
16372,0,0,0,0,...,0.00,0,0,0
16373,0,0,0,0,...,1325.31,0,0,0


In [31]:
cov.sum()

filer
3         885831.84
4        2532601.07
9          11500.00
            ...    
16371          0.00
16372          0.00
16373          0.00
dtype: float64